## Example

In [ ]:
export PGHOST=db
export PGUSER=postgres
export PGDATABASE=postgres

In [ ]:
psql << EOM
    DROP TABLE IF EXISTS seq_data;
    SELECT *
    INTO seq_data
    FROM generate_series(1,10000) a, generate_series(1,100) b;

    DROP TABLE IF EXISTS rand_data;
    SELECT a, floor(random() * 99)::int b
    INTO rand_data
    FROM generate_series(1,1000000) a;
EOM

In [ ]:
psql << EOM
    CREATE INDEX idx_seq_data_a ON seq_data(a);
    CREATE INDEX idx_rand_data_a ON rand_data(a);
EOM

## General

- Computed on ANALYZE / VACUUM / INDEX CREATION
- Used by planner to optimize the plan
- In production probably never accurate

## Types

### Default Statistics

Properties:
- Built in
- Based on one column
- Stored in both `pg_class` and `pg_statistic`
- Viewed Mainly in `pg_stats`

In [ ]:
psql << EOM
    SELECT relname, relkind, reltuples, relpages
    FROM pg_class
    WHERE relname IN ('seq_data', 'rand_data', 'idx_seq_data_a', 'idx_rand_data_a');
EOM

In [ ]:
psql << EOM
    SELECT * 
    FROM pg_stats
    WHERE tablename IN ('seq_data', 'rand_data', 'idx_seq_data_a', 'idx_rand_data_a');
EOM

##### Statistics

Name      |     Description
------    | -----
null_frac | Fraction of column entries that are null
avg_width | Average width in bytes of column's entries
n_distinct |    If greater than zero, the estimated number of distinct values in the column. If less than zero, the negative of the number of distinct values divided by the number of rows. (The negated form is used when ANALYZE believes that the number of distinct values is likely to increase as the table grows; the positive form is used when the column seems to have a fixed number of possible values.) For example, -1 indicates a unique column in which the number of distinct values is the same as the number of rows.
most_common_vals |  A list of the most common values in the column. (Null if no values seem to be more common than any others.)
most_common_freqs | A list of the frequencies of the most common values, i.e., number of occurrences of each divided by total number of rows. (Null when most_common_vals is.)
histogram_bounds |  A list of values that divide the column's values into groups of approximately equal population. The values in most_common_vals, if present, are omitted from this histogram calculation. (This column is null if the column data type does not have a < operator or if the most_common_vals list accounts for the entire population.)
correlation |   Statistical correlation between physical row ordering and logical ordering of the column values. This ranges from -1 to +1. When the value is near -1 or +1, an index scan on the column will be estimated to be cheaper than when it is near zero, due to reduction of random access to the disk. (This column is null if the column data type does not have a < operator.)
most_common_elems | A list of non-null element values most often appearing within values of the column. (Null for scalar types.)
most_common_elem_freqs |    A list of the frequencies of the most common element values, i.e., the fraction of rows containing at least one instance of the given value. Two or three additional values follow the per-element frequencies; these are the minimum and maximum of the preceding per-element frequencies, and optionally the frequency of null elements. (Null when most_common_elems is.)
elem_count_histogram |  A histogram of the counts of distinct non-null element values within the values of the column, followed by the average number of distinct non-null elements. (Null for scalar types.)

### Custom Statistics

Properties:
- Can be multi-column
- Stored in pg_statistic_ext_data
- Viewed in pg_stats_ext
- Use with caution, only when it's really needed and will be used because it's expensive

#### Functional Dependencies

##### Definition

We say that column `b` is functionally dependent on column `a` if knowledge of the value of `a` is sufficient to determine the value of `b`, that is there are no two rows having the same value of a but different values of `b`.

##### Properties

- Should not occur often in a normalized DB (can occur on denormalized DB for performance reasons)
- Can occur partially due to the variables correlation in different columns
- Can help the planner when there are filter conditions on dependent columns
- Only for equality with constants
- Can be incorrect when the constant values of the columns are not matching

##### Example

In [ ]:
psql << EOM
    DROP TABLE IF EXISTS dep_data;
    SELECT a, a + 5 b
    INTO dep_data
    FROM generate_series(1,1000000) a;

    EXPLAIN SELECT * FROM dep_data WHERE a = 10 AND b = 15;

    CREATE STATISTICS stts (dependencies) ON a, b FROM dep_data;
    ANALYZE;
    EXPLAIN SELECT * FROM dep_data WHERE a = 10 AND b = 15;
EOM

In [ ]:
psql << EOM
SELECT stxname, stxkeys, stxddependencies
  FROM pg_statistic_ext join pg_statistic_ext_data on (oid = stxoid)
  WHERE stxname = 'stts';
EOM

In [ ]:
psql << EOM
  EXPLAIN SELECT * FROM dep_data WHERE a = 10 AND b = 10;
EOM

#### Multivariate N-Distinct Counts

##### Definition

Collect n-distinct statistics for groups of columns, how much rows are existent for each combinations

##### Properties

- Only relevant for columns that are used together in GROUP BY clause

##### Example

In [ ]:
psql << EOM
    DROP TABLE IF EXISTS dist_data;
    SELECT a, a % 5 b, a % 10 c, a % 15 d
    INTO dist_data
    FROM generate_series(1,1000000) a;

    DROP STATISTICS IF EXISTS stts2;
    CREATE STATISTICS stts2 (ndistinct) ON b, c, d FROM dist_data;
    ANALYZE;

    SELECT stxkeys AS k, stxdndistinct AS nd
    FROM pg_statistic_ext join pg_statistic_ext_data on (oid = stxoid)
    WHERE stxname = 'stts2';
EOM

#### Multivariate MCV Lists

##### Definition

Multi column MCV lists, just as it sounds

##### Properties

- Good for some trends in MCV between columns
- Can be tested to see the value in the difference between base_frequency (without multi column statistic) and frequency

##### Example

In [ ]:
psql << EOM
    DROP TABLE IF EXISTS dist_data;
    SELECT a, a % 5 b, a % 10 c, a % 20 d
    INTO dist_data
    FROM generate_series(1,1000000) a;

    DROP STATISTICS IF EXISTS stts3;
    CREATE STATISTICS stts3 (mcv) ON b, c, d FROM dist_data;
    ANALYZE;

    SELECT m.* 
    FROM 
        pg_statistic_ext 
        join pg_statistic_ext_data on (oid = stxoid),
        pg_mcv_list_items(stxdmcv) m 
    WHERE stxname = 'stts3';
EOM